# 自编的模块定义根异常，以便将调用者与API相隔离

In [1]:
import logging

**示例：**当外界给函数传入无效的参数时，我们可能想抛出ValueError异常，以指出这一错误。

In [2]:
try:
    def determine_weight(volume, density):
        if density <= 0:
            raise ValueError('Density must be positive')
    
    determine_weight(1, 0)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-2-76961c76edee>", line 6, in <module>
    determine_weight(1, 0)
  File "<ipython-input-2-76961c76edee>", line 4, in determine_weight
    raise ValueError('Density must be positive')
ValueError: Density must be positive


**改进：**自己定义一套新的异常体系，提供一种根异常，然后令该模块所抛出的其他异常，都继承这个根异常。

In [3]:
class Error(Exception):
    """Base-class for all exceptions raised by this module."""

class InvalidDensityError(Error):
    """There was a problem with a provided density value."""

采用try/except语句来捕获根异常

In [4]:
class my_module(object):
    Error = Error
    InvalidDensityError = InvalidDensityError

    @staticmethod
    def determine_weight(volume, density):
        if density <= 0:
            raise InvalidDensityError('Density must be positive')
try:
    weight = my_module.determine_weight(1, -1)
    assert False
except my_module.Error as e:
    logging.error('Unexpected error: %s', e)

ERROR:root:Unexpected error: Density must be positive


**调用代码与API隔离的好处：**  
1. 通过捕获根异常，调用者得知他们在使用你的API时，所编写的调用代码是否正确。

In [5]:
weight = 5
try:
    weight = my_module.determine_weight(1, -1)
    assert False
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error('Bug in the calling code: %s', e)

assert weight == 0

2. 帮助模块开发者寻找API里面的bug

In [6]:
weight = 5
try:
    weight = my_module.determine_weight(1, -1)
    assert False
except my_module.InvalidDensityError:
    weight = 0
except my_module.Error as e:
    logging.error('Bug in the calling code: %s', e)
except Exception as e:
    logging.error('Bug in the API code: %s', e)
    raise

assert weight == 0

3. 便于API的后续演化，将来可能在API里面提供更为具体的异常。

In [7]:
class NegativeDensityError(InvalidDensityError):
    """A provided density value was negative."""

def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError

In [8]:
try:
    my_module.NegativeDensityError = NegativeDensityError
    my_module.determine_weight = determine_weight
    try:
        weight = my_module.determine_weight(1, -1)
        assert False
    except my_module.NegativeDensityError as e:
        raise ValueError('Must supply non-negative density') from e
    except my_module.InvalidDensityError:
        weight = 0
    except my_module.Error as e:
        logging.error('Bug in the calling code: %s', e)
    except Exception as e:
        logging.error('Bug in the API code: %s', e)
        raise
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-8d24802704bf>", line 5, in <module>
    weight = my_module.determine_weight(1, -1)
  File "<ipython-input-7-a904bdae8298>", line 6, in determine_weight
    raise NegativeDensityError
NegativeDensityError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-8-8d24802704bf>", line 8, in <module>
    raise ValueError('Must supply non-negative density') from e
ValueError: Must supply non-negative density


In [9]:
class WeightError(Error):
    """Base-class for weight calculation errors."""

class VolumeError(Error):
    """Base-class for volume calculation errors."""

class DensityError(Error):
    """Base-class for density calculation errors."""